In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import PIL
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import random
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf

from numpy import save
from numpy import load

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import Input, Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import to_categorical

In [2]:
def plot_images(dataset, n_images, samples_per_image):
    output = np.zeros((32 * n_images, 32 * samples_per_image, 3))

    row = 0
    for images in dataset.repeat(samples_per_image).batch(n_images):
        output[:, row*32:(row+1)*32] = np.vstack(images.numpy())
        row += 1

    plt.figure()
    plt.imshow(output)
    plt.show()

def flip(x: tf.Tensor) -> tf.Tensor:
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)

    return x

def color(x: tf.Tensor) -> tf.Tensor:
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

def rotate(x: tf.Tensor) -> tf.Tensor:
    return tf.image.rot90(x, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

def zoom(x: tf.Tensor) -> tf.Tensor:
    # Generate 20 crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize([img], boxes=boxes, box_ind=np.zeros(len(scales)), crop_size=(32, 32))
        # Return a random crop
        return crops[tf.random_uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]


    choice = tf.random.uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))

In [3]:
X = load('G:/Desktop/intel image classification/keras_train_x_256(3).npy')
y = load('G:/Desktop/intel image classification/keras_train_y_256(3)(all).npy')

In [4]:
x_test = load('G:/Desktop/intel image classification/keras_test_x_256(3).npy')
y_test = load('G:/Desktop/intel image classification/keras_test_y_256(3)(all).npy')

In [5]:
X.shape

(14034, 256, 256, 3)

In [5]:
dataset = tf.data.Dataset.from_tensor_slices(X[0:10])

In [6]:
dataset

<TensorSliceDataset shapes: (244, 244, 3), types: tf.float16>

In [7]:
X[0:10].shape

(10, 244, 244, 3)

In [22]:
dataset.sample()

AttributeError: 'TensorSliceDataset' object has no attribute 'sample'

In [15]:
# Add augmentations
augmentations = [flip, color, zoom, rotate]

# Add the augmentations to the dataset
dataset_aug = dataset.map(lambda x: color(x))

In [17]:
dataset

<MapDataset shapes: (244, 244, 3), types: tf.float16>

In [16]:
temp = list(dataset.as_numpy_iterator())

In [17]:
temp1 = list(dataset_aug.as_numpy_iterator())

In [18]:
len(temp1)

10

In [19]:
temp

[array([[[0.204  , 0.2903 , 0.2705 ],
         [0.2235 , 0.306  , 0.2864 ],
         [0.2432 , 0.3176 , 0.306  ],
         ...,
         [0.9883 , 0.9688 , 0.957  ],
         [0.953  , 0.9253 , 0.9453 ],
         [0.9253 , 0.894  , 0.933  ]],
 
        [[0.1647 , 0.2354 , 0.2235 ],
         [0.1765 , 0.2393 , 0.2274 ],
         [0.1921 , 0.2432 , 0.2354 ],
         ...,
         [0.8276 , 0.8115 , 0.8076 ],
         [0.89   , 0.8706 , 0.8784 ],
         [0.9414 , 0.9175 , 0.933  ]],
 
        [[0.1137 , 0.1686 , 0.1608 ],
         [0.1177 , 0.153  , 0.149  ],
         [0.1216 , 0.1372 , 0.1333 ],
         ...,
         [0.596  , 0.5845 , 0.5923 ],
         [0.8    , 0.792  , 0.7803 ],
         [0.957  , 0.949  , 0.9253 ]],
 
        ...,
 
        [[0.0392 , 0.0353 , 0.04315],
         [0.0392 , 0.0353 , 0.04315],
         [0.0392 , 0.0353 , 0.04315],
         ...,
         [0.0902 , 0.1372 , 0.1137 ],
         [0.0863 , 0.1333 , 0.1098 ],
         [0.0863 , 0.1294 , 0.1059 ]],
 
     

In [20]:
temp1

[array([[[0.1914 , 0.3086 , 0.2383 ],
         [0.21   , 0.3218 , 0.2542 ],
         [0.2303 , 0.332  , 0.2783 ],
         ...,
         [0.9033 , 0.867  , 0.867  ],
         [0.8696 , 0.8413 , 0.877  ],
         [0.826  , 0.8096 , 0.8594 ]],
 
        [[0.1647 , 0.2612 , 0.2095 ],
         [0.1783 , 0.2646 , 0.2168 ],
         [0.197  , 0.268  , 0.2312 ],
         ...,
         [0.7656 , 0.744  , 0.748  ],
         [0.8193 , 0.798  , 0.8174 ],
         [0.862  , 0.836  , 0.867  ]],
 
        [[0.128  , 0.2039 , 0.1653 ],
         [0.1401 , 0.1904 , 0.167  ],
         [0.1522 , 0.1769 , 0.1635 ],
         ...,
         [0.566  , 0.5557 , 0.5703 ],
         [0.741  , 0.7256 , 0.7197 ],
         [0.8765 , 0.8545 , 0.839  ]],
 
        ...,
 
        [[0.08276, 0.08594, 0.09546],
         [0.08276, 0.08594, 0.09546],
         [0.08276, 0.08594, 0.09546],
         ...,
         [0.1112 , 0.1769 , 0.12195],
         [0.1079 , 0.1736 , 0.11865],
         [0.1096 , 0.1702 , 0.1172 ]],
 
     